In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # load the files
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data"))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline

In [ ]:
# Let's start by loading the data and seeing what we have
train_path = '../data/train.json'
with open(train_path) as json_data:
    train_data = json.load(json_data)

print(train_data.keys())

In [ ]:
labels = train_data['annotations']
images = train_data['images']

# Labels is a list with dictionaries that contains the image ID as well as the tags associated with that image
# images is a list. Each entry is a dictionary that has an image ID and a link to the image being classified

In [ ]:
#First let's do some exploration with the labels. 
# I'm going to put this in a dataframe.
image_no = []
image_labels = []
count = 0
for entry in labels:  
    imageId = entry['imageId']
    imageLabels = entry['labelId']
    for lab in imageLabels:
        image_no.append(imageId)
        image_labels.append(lab)
        
    count+=1
    

In [ ]:
train_df = pd.DataFrame(np.array([image_no, image_labels]).T, columns = ['imageId', 'labelId'])
train_df['imageId'].value_counts()

In [ ]:
# I find this to be a more useful form of the data, so I'm going to save it like this
train_df.to_csv('../data/train_images_lables.csv')

In [ ]:
train_df = pd.read_csv('../data/train_images_lables.csv')

In [ ]:
print('There are ', len(train_df), 'image and label pairs in this data set')

In [ ]:
label_counts = train_df['labelId'].value_counts()
plt.scatter(label_counts.index.values, label_counts.values)
plt.xlabel('Label')
plt.ylabel('Number of occurences')
plt.yscale('log')
print('Median label occurence', np.median(label_counts.values))
print('Minimum label occurence', np.min(label_counts.values))
print('Maximum label occurence ', np.max(label_counts.values))
print('There are ', len(label_counts), ' labels in this dataset')

In [ ]:
image_counts = train_df['imageId'].value_counts()
plt.scatter(image_counts.index.values, image_counts.values)
plt.xlabel('Image #')
plt.ylabel('Number of labels')
print('Median label occurence', np.median(image_counts.values))
print('Minimum label occurence', np.min(image_counts.values))
print('Maximum label occurence ', np.max(image_counts.values))
print('There are ', len(image_counts), ' images in this dataset')

In [ ]:
plt.hist(image_counts.values)
plt.xlabel('Number of labels on image')
plt.show()

In [ ]:
# taking the log here because of the outliers
# most labels are assigned to e^10 images or about 22K images. 
# The most common label has 743250 which is about right with e^13.5 in this histogram
plt.hist(np.log(label_counts.values))
plt.xlabel('ln(Number of images with one label)')
plt.show()

In [ ]:
# Let's get all the images and save them...
image_urls = train_data['images']

In [ ]:
# take a look at what this looks like
image_urls[0]

In [ ]:
import urllib.request as urllib


In [ ]:
for entry in image_urls:
    try:
        url = entry['url']
        imageId = entry['imageId']
        urllib.urlretrieve(url, '../data/images/img_' + imageId + '.png')
    except:
        print(imageId)

In [ ]:
# One of the critical steps in training a neural network to recognize labels in images will be to 
# crop the images so they contain only the objects we want to identify
# as a first pass,we are going to use the tensorflow object detection API, found here: 
# https://github.com/tensorflow/models/tree/master/research/object_detection/
import glob
image_directory = '../data/images/'

In [ ]:
images = glob.glob(image_directory + '*.png')

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from PIL import Image

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

from matplotlib import pyplot as plt

### Notes on the preliminary plan, which may not make sense.

We could use tensor flow's object detection API, but unforunately clothing is not a class there. A rough first pass could use the out of the box object detector, which has the output shown below.

We will instead try training a model using this tutorial : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_pets.md

and the deep fasions data set : http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html

to train a neural network to find the bounding boxes of our fasion pieces (we'll only go for the big objects, not things like logos). 

After we find the bouding boxes, we will train a second neural network to tag the images with the labels from the imaterialist dataset.

## The following is copied from the object detection api notebook

In [ ]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
# ssd_mobilenet_v1_coco_2017_11_17
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/Users/lisa/.pyenv/versions/3.6.0/envs/imaterialist/lib/python3.6/site-packages/tensorflow/models/research/object_detection/data/', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

print(PATH_TO_LABELS)

In [ ]:
# download model
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    print(file_name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())
        print(os.getcwd())

In [ ]:
# load tensorflow model into memory
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
import glob
PATH_TO_TEST_IMAGES_DIR = '../data/images/'
TEST_IMAGE_PATHS = glob.glob(PATH_TO_TEST_IMAGES_DIR + '*.png')

print(TEST_IMAGE_PATHS)
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
for i in range(10):
    image_path = TEST_IMAGE_PATHS[i+10]
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    plt.close()